In [1]:
import os
import cv2
import numpy as np

from torchvision import transforms as T
import torch

import matplotlib.pyplot as plt
!pip install mean_average_precision
from mean_average_precision import MeanAveragePrecision

import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

  Created wheel for mean-average-precision: filename=mean_average_precision-0.0.2.1-py3-none-any.whl size=8230 sha256=4be26799d3bf8d50b0921c1556e213e3f12f5dfac138df20f7d3d4a3c63ac810
  Stored in directory: /root/.cache/pip/wheels/c7/47/2a/2abd387baa007d102596e6b992d53d475a824eb685926b166d
Successfully built mean-average-precision
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
class Apple_Dataset(object):
    def __init__(self, root_dir, transforms):
        self.root_dir = root_dir
        self.transforms = transforms

        # Load all image and mask files, sorting them to ensure they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root_dir, "images"))))
        self.masks = list(sorted(os.listdir(os.path.join(root_dir, "masks"))))

    def __getitem__(self, idx):
        # Load images and masks
        img_path = os.path.join(self.root_dir, "images", self.imgs[idx])
        mask_path = os.path.join(self.root_dir, "masks", self.masks[idx])

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (img.shape[1]//2, img.shape[0]//2))
        mask = cv2.imread(mask_path, 0)

        obj_ids = np.unique(mask)
        num_apples = len(obj_ids) - 1

        # Remove Background
        obj_ids = obj_ids[1:]

        # Split the color-encoded masks into a set of binary masks
        masks = mask == obj_ids[:, None, None]

        # Get bbox coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        h, w = mask.shape
        for each in range(num_objs):
            pos = np.where(masks[each])
            xmin = np.min(pos[1])//2
            xmax = np.max(pos[1])//2
            ymin = np.min(pos[0])//2
            ymax = np.max(pos[0])//2

            if xmin == xmax or ymin == ymax:
                continue

            xmin = np.clip(xmin, a_min=0, a_max=w)
            xmax = np.clip(xmax, a_min=0, a_max=w)
            ymin = np.clip(ymin, a_min=0, a_max=h)
            ymax = np.clip(ymax, a_min=0, a_max=h)
            boxes.append([xmin, ymin, xmax, ymax])

        # Convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)

        # There is only one class (apples)
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        # All instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        target["count"] = num_apples

#         if self.transforms is not None:
#             img, target = self.transforms(img, target)

        img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [3]:
class Utils(object):
    def __init__(self):
        pass
    
    def get_transform(self, train):
        transforms = [T.ToTensor()]
        # if train:
        #     transforms.append(T.RandomHorizontalFlip(0.5))
        return T.Compose(transforms)
    
    def get_count(self, boxes, scores):
        boxes = np.array(boxes, dtype=int)
        count = 0
        for box, score in zip(boxes, scores):
            if score > 0.9:
                count += 1

        return count


    def my_collate_fn(self, batch):
        images = [each[0] for each in batch]
        targets = [each[1] for each in batch]
        return images, targets


    def generate_bbox(self, img, boxes):
        for (x_min, y_min, x_max, y_max) in boxes:
            cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            
    def get_frcnn_model_instance(self, num_classes):
        # load an instance segmentation model pre-trained pre-trained on COCO
        model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)

        # get number of input features for the classifier
        in_features = model.roi_heads.box_predictor.cls_score.in_features
        # replace the pre-trained head with a new one
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
        return model

In [4]:
root_path = '../input/minneapple/Dataset/Dataset'
out_folder = '/kaggle/working/apple_counting/'
model_path = '../input/apple-counting-rcnn/apple_counting/Models/FRCNN_30Dec2020_best.pt'

util = Utils()
test_dataset = Apple_Dataset(os.path.join(root_path, 'test'), util.get_transform(True))
data_loader = {'test': torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=True, num_workers=1, collate_fn=util.my_collate_fn)}

device = torch.device("cpu" if not torch.cuda.is_available() else "cuda:0")

model = util.get_frcnn_model_instance(num_classes=2)
model.load_state_dict(torch.load(model_path, map_location=device))
model = model.to(device)
model.eval()

category = 'test'
metric_fn = MeanAveragePrecision(num_classes=1)
precision_list = []

for i, (images, targets) in enumerate(data_loader[category]):
    images = list(image.to(device) for image in images)
    y_pred = model(images)
    
    gt = targets[0]['boxes'].numpy()
    gt = np.append(gt, np.zeros((len(gt),3)), axis=1)
        
    boxes_pred = np.array(y_pred[0]['boxes'].cpu().detach().numpy(), dtype=int)
    scores_pred = np.array(y_pred[0]['scores'].cpu().detach().numpy(), dtype=float)
    
    num_apples = targets[0]['count']
    num_apples_pred = util.get_count(boxes_pred, scores_pred)
    precision = min(num_apples, num_apples_pred)/num_apples
    precision_list.append(precision)
            
    preds = np.column_stack([boxes_pred[:,0], boxes_pred[:,1], boxes_pred[:,2], boxes_pred[:,3], np.zeros((len(boxes_pred), 1), dtype=int), scores_pred])
    
    metric_fn.add(preds, gt)
    
print("AP @ IoU=0.75 : {}".format(metric_fn.value(iou_thresholds=0.75)['mAP']))
print("AP @ IoU=0.5 : {}".format(metric_fn.value(iou_thresholds=0.5)['mAP']))
print("AP @ IoU=0.5:0.05:0.95 : {}".format(metric_fn.value(iou_thresholds=np.arange(0.5, 0.95, 0.05))['mAP']))
print ("Count Accuracy: {}".format(sum(precision_list)/len(precision_list)))

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth



AP @ IoU=0.75 : 0.29007238149642944
AP @ IoU=0.5 : 0.7669376730918884
AP @ IoU=0.5:0.05:0.95 : 0.40526947379112244
Count Accuracy: 0.6603662575830639
